# Análisis NLP de reseñas de alimentos

## Librerías importadas

In [30]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

## Carga de los datos preprocesados

In [31]:
df = pd.read_csv('ReviewsPreprocessingStemming.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148845 entries, 0 to 148844
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Score   148845 non-null  int64 
 1   Text    148845 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [44]:
df_l = pd.read_csv('ReviewsPreprocessingLematization.csv')
df_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148845 entries, 0 to 148844
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Score   148845 non-null  int64 
 1   Text    148845 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


## Extracción de matriz término-documento y clasificación

### Preprocesado mediante Stemming

En primer lugar, se divide el conjunto de datos original en entrenamiento, test y validación. También se define una pequeña función que facilitaría el uso de distitnos clasificadores más adelante.

In [32]:
df_train, df_test = train_test_split(df, train_size=0.8)
df_train, df_val = train_test_split(df, train_size=0.8)

print(f'Train size: {len(df_train)}, Test size: {len(df_test)}, Validation size: {len(df_val)}')

Train size: 119076, Test size: 29769, Validation size: 29769


In [33]:
def classify_with(classifier, X_train, y_train, X_test, y_test):
    classifier.fit(X_train, y_train)
    y_predicted = classifier.predict(X_test)
    return {
        'accuracy_score': accuracy_score(y_test, y_predicted),
        'recall_score': recall_score(y_test, y_predicted, average='macro'),
        'precision_score': precision_score(y_test, y_predicted, average='macro')
    }

En esta parte, se realiza la primera matriz término documento, las cuales representan la frecuencia de las palabras encontradas en los textos. En este caso, se ua CountVectorizer.

In [34]:
tf_vectorizer = CountVectorizer()
Xtrain = tf_vectorizer.fit_transform(df_train['Text'])
Xtest = tf_vectorizer.transform(df_test['Text'])
Xval = tf_vectorizer.transform(df_val['Text'])

encoder = LabelEncoder()
ytrain = encoder.fit_transform(df_train['Score']) 
ytest = encoder.transform(df_test['Score'])
yval = encoder.transform(df_val['Score'])

Para la clasificación, se consideró el uso de un RandomForest, debido a su buenos resultados en la literatura.

In [37]:
rf = RandomForestClassifier(verbose=1, n_jobs=-1)
results = classify_with(rf, Xtrain, ytrain, Xtest, ytest)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.9s finished


Los resultados obtenidos se muestran a continuación.

In [38]:
results

{'accuracy_score': 0.6308240115556452,
 'recall_score': 0.630429582505654,
 'precision_score': 0.6332420949816501}

A continuación, se realiza la segunda matriz término documento. La diferencia más destacable de TF-IDF con respecto a CountVectorizer es que su representación de las frecuencias está normalizada.

In [39]:
tfidf_vectorizer = TfidfVectorizer()
Xtrain = tfidf_vectorizer.fit_transform(df_train['Text'])
Xtest = tfidf_vectorizer.transform(df_test['Text'])
Xval = tfidf_vectorizer.transform(df_val['Text'])

encoder = LabelEncoder()
ytrain = encoder.fit_transform(df_train['Score'])
ytest = encoder.transform(df_test['Score'])
yval = encoder.transform(df_val['Score'])

In [40]:
rf_tfidf = RandomForestClassifier(verbose=1, n_jobs=-1)
results_tfidf = classify_with(rf_tfidf, Xtrain, ytrain, Xtest, ytest)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.5min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


Los resultados obtenidos se muestran a continuación.

In [41]:
results_tfidf

{'accuracy_score': 0.6308240115556452,
 'recall_score': 0.6303646113516179,
 'precision_score': 0.6343812713257779}

### Preprocesado mediante lematización

En primer lugar, se divide el conjunto de datos original en entrenamiento, test y validación. También se define una pequeña función que facilitaría el uso de distitnos clasificadores más adelante.

In [49]:
df_l_train, df_l_test = train_test_split(df_l, train_size=0.8)
df_l_train, df_l_val = train_test_split(df_l, train_size=0.8)

print(f'Train size: {len(df_l_train)}, Test size: {len(df_l_test)}, Validation size: {len(df_l_val)}')

Train size: 119076, Test size: 29769, Validation size: 29769


In [50]:
tf_vectorizer = CountVectorizer()
Xtrain = tf_vectorizer.fit_transform(df_l_train['Text'])
Xtest = tf_vectorizer.transform(df_l_test['Text'])
Xval = tf_vectorizer.transform(df_l_val['Text'])

encoder = LabelEncoder()
ytrain = encoder.fit_transform(df_l_train['Score']) 
ytest = encoder.transform(df_l_test['Score'])
yval = encoder.transform(df_l_val['Score'])

In [51]:
rf_l = RandomForestClassifier(verbose=1, n_jobs=-1)
results_l = classify_with(rf_l, Xtrain, ytrain, Xtest, ytest)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


In [52]:
results_l

{'accuracy_score': 0.9235782189526017,
 'recall_score': 0.9235264100386358,
 'precision_score': 0.9240594434649664}

In [53]:
tfidf_vectorizer = TfidfVectorizer()
Xtrain = tfidf_vectorizer.fit_transform(df_l_train['Text'])
Xtest = tfidf_vectorizer.transform(df_l_test['Text'])
Xval = tfidf_vectorizer.transform(df_l_val['Text'])

encoder = LabelEncoder()
ytrain = encoder.fit_transform(df_l_train['Score'])
ytest = encoder.transform(df_l_test['Score'])
yval = encoder.transform(df_l_val['Score'])

In [54]:
rf_tfidf_l = RandomForestClassifier(verbose=1, n_jobs=-1)
results_tfidf_l = classify_with(rf_tfidf_l, Xtrain, ytrain, Xtest, ytest)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.2min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


In [55]:
results_tfidf_l

{'accuracy_score': 0.9233766670025866,
 'recall_score': 0.9233117543692307,
 'precision_score': 0.9240272489312658}

## Conclusiones

A través de los resultados obtenidos en este notebook, se pueden realizar las siguientes afirmaciones:

- El preprocesado mediante lematización da claramente mejores resultados con respecto al stemming, siendo la precisión de un 92% para el primero y de un 62% para el segundo.
- La diferencia de usar la vectorización mediante el CountVectorizer y el TF-IDF es muy poca o nula.
- A pesar de no estar reflejado en el notebook, el tiempo de ejecución de RandomForest comparado con SVC es bastante inferior. Este último se tuvo que descartar debido a su larga duración de excución.

Por otra parte, cabe destacar que, durante las evaluaciones de distintos hiperparámetros para los clasificadores, no se obtuvo mejoras notables (~1%) con respecto a los definido por defecto. Sin embargo, su tiempo de ejecución si que aumenta considerablemente coomo, por ejemplo, el uso de n_estimators = 100, que tarda 8 minutos, frente a n_estiamtors = 500, que duraba más de 30 minutos.
  